In [75]:
import os
import numpy as np
from scipy.optimize import minimize
from simsopt.geo import SurfaceRZFourier
from simsopt.objectives import SquaredFlux
from simsopt.objectives import QuadraticPenalty
from simsopt.geo import curves_to_vtk
from simsopt.field import BiotSavart, Current, coils_via_symmetries, Coil
from simsopt.geo import (
    CurveLength, CurveCurveDistance,
    MeanSquaredCurvature, LpCurveCurvature, CurveCWSFourier, ArclengthVariation, CurveXYZFourier
)

import matplotlib.pyplot as plt

In [76]:
wout = '../input.axiTorus_nfp3_QA_final'

ncoils = 1
order = 1 # order of dofs of cws curves
quadpoints = 300 #13 * order
ntheta = 50
nphi = 42
ext_via_normal_factor = 0.2565

In [77]:
phi = np.linspace(0, 1, 256, endpoint=True)
theta = np.linspace(0, 1, 256, endpoint=True)

In [78]:
# CREATE COIL WINDING SURFACE SURFACE
cws = SurfaceRZFourier.from_vmec_input(wout, range="half period", quadpoints_theta = theta, quadpoints_phi = phi)
cws_full = SurfaceRZFourier.from_vmec_input(wout, range="full torus", quadpoints_theta = theta, quadpoints_phi = phi)

# cws.extend_via_normal(ext_via_normal_factor)
# cws_full.extend_via_normal(ext_via_normal_factor)

# CREATE CURVES + COILS     
base_curves = []


poloidal_curve = CurveCWSFourier(mpol=cws.mpol, ntor=cws.ntor, idofs=cws.x,
                                quadpoints=quadpoints, order=order, nfp=cws.nfp, stellsym=cws.stellsym)
poloidal_curve.set_dofs([1, 0, 0, 0, 0, 0, 0, 0])
base_curves.append(poloidal_curve)

saddle_curve = CurveCWSFourier(mpol=cws.mpol, ntor=cws.ntor, idofs=cws.x,
                                quadpoints=quadpoints, order=order, nfp=cws.nfp, stellsym=cws.stellsym)
saddle_curve.set_dofs([0, np.pi/2, 0.7, 0, 0, np.pi, 0, 0.5])
base_curves.append(saddle_curve)

In [79]:
OUT_DIR = "./papercoils/"
os.makedirs(OUT_DIR, exist_ok=True)


curves = base_curves
curves_to_vtk(curves, OUT_DIR + "curves_init", close=True)
cws_full.to_vtk(OUT_DIR + "cws_init")
